In [866]:
import numpy as np
import pandas as pd
import random as rnd

In [867]:
data = pd.read_csv("C:\\Users\\elad\\Desktop\\Kaggle\\Load prediction\\train.csv")
data = np.array(data)
np.random.shuffle(data)

In [868]:
# Data completion
dataSize = data.shape[0]

for i in range(dataSize):
    if data[i,12] == 'N':
        data[i,12] = 0
    elif data[i,12] == 'Y':
        data[i,12] = 1
    else:
        print('WARNING IN DATA\'s CLASS')


# Co Applicant Income
counter = 0
for x in data[:,7]:
    if x > 0:
        counter += 1

medianCoApplicantIncome = int(np.sum(data[:,7]) / counter) + 0.0

for i in range(len(data[:,7])):
    if data[i,7] == 0:
        data[i,7] = medianCoApplicantIncome

        
# Loan Amount
counter = 0
sum = 0
for x in data[:,8]:
    if not np.isnan(x):
        counter += 1
        sum += x

medianLoanAmount = int(sum / counter) + 0.0
for i in range(len(data[:,8])):
    if np.isnan(data[i,8]):
        data[i,8] = medianLoanAmount
        

In [869]:
# Cleaning Data Calculations
maxApplicantIncome = 20000
minApplicantIncome = np.min(data[:,6])
maxCoApplicantIncome = 8000
minCoApplicantIncome = np.min(data[:,7])
maxLoanAmount = 450
minLoanAmount = np.min(data[:,8])

def clean_sample(sample):
    #Gender
    if sample[1] == 'Male':
        sample[1] = 0
    elif sample[1] == 'Female':
        sample[1] = 1
    else:
        sample[1] = 0
        if rnd.randint(1,5) == 1:
            sample[1] = 1
            
    
    #Married
    if sample[2] == 'No':
        sample[2] = 0
    elif sample[2] == 'Yes':
        sample[2] = 1
    else:
        sample[2] = rnd.randint(0,1)
      
    #Dependents
    if sample[3] == '0':
        sample[3] = 0.0
    elif sample[3] == '1':
        sample[3] = 1.0
    elif sample[3] == '2':
        sample[3] = 2.0
    elif sample[3] == '3+':
        sample[3] = 3.0
    else:
        if rnd.randint(1,4) != 1:
            sample[3] = 0 
        else:
            sample[3] = rnd.randint(1,3)
    
    #Graduated
    if sample[4] == 'Not Graduate':
        sample[4] = 0
    elif sample[4] == 'Graduate':
        sample[4] = 1
    else:
        sample[4] = rnd.randint(0,1)
        
    #Self-Employed
    if sample[5] == 'No':
        sample[5] = 0
    elif sample[5] == 'Yes':
        sample[5] = 1
    else:
        sample[5] = 0
        if rnd.randint(1,4) == 1:
            sample[5] = 1
        
    #Applicant Income
    sample[6] = (sample[6] - minApplicantIncome) / (maxApplicantIncome - minApplicantIncome)
    
    #Co Applicant Income
    sample[7] = (sample[7] - minCoApplicantIncome) / (maxCoApplicantIncome - minCoApplicantIncome)
    
    # Loan Amount
    sample[8] = (sample[8] - minLoanAmount) / (maxLoanAmount - minLoanAmount)
    
    # Loan Amount Term
    if sample[9] <= 120:
        sample[9] = 0
    elif sample[9] == 180 or sample[9] == 240:
        sample[9] = 1
    elif sample[9] == 360:
        sample[9] = 2
    elif sample[9] == 480:
        sample[9] = 3
    elif sample[9] == 300:
        sample[9] = 4
    else:
        sample[9] = 2
        if rnd.randint(1,4) == 1:
             sample[9] = [0,1,3,4][rnd.randint(0,3)]
        
   
    # Credit History
    if np.isnan(sample[10]):
        sample[10] = 1
        if len(sample) == 13:
            sample[10] = sample[12]
        else:
            if rnd.randint(1,5) == 1:
                sample[10] = 0
            
    # Property Area
    if sample[11] == 'Semiurban':
        sample[11] = 0
    elif sample[11] == 'Urban':
        sample[11] = 1
    elif sample[11] == 'Rural':
        sample[11] = 2
    else:
        print("ERROR IN PROPERY HISTORY")
    
    temp = []
    
    leng = 11
    for i in range(1,1+leng):
        for j in range(1,1+leng):
            temp.append(sample[i]*sample[j])
    
    return np.array(temp)

In [870]:
newData = np.array([clean_sample(x) for x in data])
#data = np.delete(data,3,1)

In [871]:
n = 520
f_n = 121
X_train = newData[:n]
Y_train = np.array(data[:n,12], dtype='int')
X_test = newData[n:]
Y_test = np.array(data[n:,12], dtype='int')

In [872]:
from sklearn.svm import SVC

In [873]:
# Linear SVC
linear_svc = SVC(degree=3)
linear_svc.fit(X_train, Y_train)
acc_linear_svc = linear_svc.score(X_train, Y_train)
print("Linear SVC Train accuracy :  " + str(acc_linear_svc))
acc_linear_svc = linear_svc.score(X_test, Y_test)
print("Linear SVC Test accuracy :  " + str(acc_linear_svc))

Linear SVC Train accuracy :  0.8326923076923077
Linear SVC Test accuracy :  0.851063829787234


In [803]:
test = pd.read_csv("C:\\Users\\elad\\Desktop\\Kaggle\\Load prediction\\test.csv")
test = np.array(test)
np.random.shuffle(test)

In [804]:
# Data completion
dataSize = test.shape[0]


for i in range(len(test[:,7])):
    if test[i,7] == 0:
        test[i,7] = medianCoApplicantIncome


for i in range(len(test[:,8])):
    if np.isnan(test[i,8]):
        test[i,8] = medianLoanAmount
        

In [805]:
test = np.array([clean_sample(x) for x in test])

In [806]:
test_prediction = linear_svc.predict(test)

ValueError: X.shape[1] = 100 should be equal to 121, the number of features at training time